### 1. ResNet 

In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten
from keras.layers import Dropout, BatchNormalization, Activation, add
from keras.optimizers import Adam
# ResNet单元的构建
def Res_layer(x, filters, pooling=False, dropout=0.0):
    temp = x
    # 第一个卷积层
    temp = Conv2D(filters, (3,3), padding="same")(temp)
    temp = BatchNormalization()(temp)
    temp = Activation("relu")(temp)
    # 第二个卷积层
    temp = Conv2D(filters, (3,3), padding="same")(temp)
    # 将x与第二个卷积层的输出相加
    x = add([temp, Conv2D(filters, (3,3), padding="same")(x)])
    if pooling:
        x = MaxPooling2D((2,2))(x)
    if dropout != 0.0:
        x = Dropout(dropout)(x)
    x = BatchNormalization()(x)
    # 将最后的结果使用ReLu激活函数进行处理
    outputs = Activation("relu")(x)
    return outputs

In [ ]:
inputs = Input(shape = (100, 100, 3))
x = Res_layer(inputs, 32, pooling=True, dropout=0.3)
x = Res_layer(x, 32, pooling=True, dropout=0.3)
x = Res_layer(x, 64, pooling=True, dropout=0.3)
x = Res_layer(x, 64, pooling=True, dropout=0.3)
x = Res_layer(x, 128, pooling=True, dropout=0.3)
x = Res_layer(x, 128, pooling=True, dropout=0.3)
x = Flatten()(x)
x = Dropout(0.5)(x)
outputs = Dense(1, activation = "sigmoid")(x)
resnet_model = Model(inputs, outputs)

In [ ]:
resnet_model.compile(loss='binary_crossentropy',
                     optimizer=Adam(),
                     metrics=['accuracy'])
resnet_model.fit(X_train, 
                 y_train, 
                 epochs=20, 
                 batch_size=64, 
                 verbose=2,
                 validation_split=0.2)

### 2. Inception网络

In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Dense
from keras.layers import Flatten, Dropout, concatenate
from keras.optimizers import Adam

In [ ]:
def Inception_Layer(inputs):
    # 第一部分
    layer1 = Conv2D(filters=32,
                    kernel_size=(1, 1),
                    strides=1,
                    padding='same',
                    activation='relu')(inputs)
    # 第二部分
    layer2 = Conv2D(filters=32,
                    kernel_size=(1, 1),
                    strides=1,
                    padding='same',
                    activation='relu')(inputs)
    layer2 = Conv2D(filters=64,
                    kernel_size=(3, 3),
                    strides=1,
                    padding='same',
                    activation='relu')(layer2)
    # 第三部分
    layer3 = Conv2D(filters=32,
                    kernel_size=(1, 1),
                    strides=1,
                    padding='same',
                    activation='relu')(inputs)
    layer3 = Conv2D(filters=64,
                    kernel_size=(5, 5),
                    strides=1,
                    padding='same',
                    activation='relu')(layer3)
    # 第四部分
    layer4 = MaxPooling2D(pool_size=(3, 3),
                          strides=1,
                          padding='same')(inputs)
    layer4 = Conv2D(filters=64,
                    kernel_size=(1, 1),
                    strides=1,
                    padding='same',
                    activation='relu')(layer4)
    # 将所有部分的输出结果拼接起来
    layers = [layer1, layer2, layer3, layer4]
    outputs = concatenate(layers, axis=3)
    return outputs

In [ ]:
inputs = Input(shape = (100, 100, 3))
x = Inception_Layer(inputs)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
outputs = Dense(1, activation="sigmoid")(x)
inception_model = Model(inputs, outputs)
inception_model.compile(loss='binary_crossentropy',
                     optimizer=Adam(),
                     metrics=['accuracy'])
inception_model.fit(X_train,
                    y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)